In [ ]:
# First, import revelant packages to assist in data extraction and formatting in Python.
import pandas as pd
import numpy as np
import os
import datetime
import glob
from pathlib import Path

In [ ]:
# Our team downloaded the 27 CSV files which contained survey responses from April 2020 to March 2021. After downloading these files into a folder, we 
# parsed through them and combined them into a single dataframe in Python. 

# First, we ensured that all of the correct CSVs were in the specified folder, by listing all the CSV file in the directory. Actual directory path removed to maintain anonymity.
path = r''
for filename in glob.glob(path):
    data = pd.read_csv(filename)
    df = pd.DataFrame(data)
    print(filename)

In [ ]:
# After verifying that all the correct files were in our destination folder, we proceeded to combine all of the CSVs into one Pandas Dataframe.

# Specify Path in which you have saved the Census CSV files. Actual path name removed to maintain anonymity.
path = r''

# Combine CSVs into one dataframe.
df = pd.concat([pd.read_csv(f, encoding='latin1') for f in glob.glob(path)],ignore_index=True)

In [ ]:
# After saving the data into a single Pandas dataframe, we exported the data into one large CSV file & Parquet file so that our team members could download and use in Python and Tableau.
df.to_csv('Full Dataset2.csv', index=False, mode = 'a')

# Dataframe was saved to Parquet in our shared Google Drive. Actual Google Drive link removed to maintain anonymity.
df.to_parquet('')

In [ ]:
# After saving our Combined Dataframe into a CSV file, our team placed that folder in our Google Drive, and mounted our Drive to Colab, so that all members of the team
# could access the file. Mounting the Drive allows our team to connect to & import files that are stored in our shared Google Drive. 

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# After mounting the Drive, our team read the Parquet file which contained data for all survey weeks into Colab. 
data = pd.read_parquet('/content/drive/Shareddrives/GCP-UCDavis/data/full_dataset.parquet')

# We were now ready to move onto the Data Preprocessing stage.

# Data Preprocessing

In [ ]:
# First, while the survey starting and ending date was known to our team, it was listed in the raw data as only a week number, and was not correctly formatted for analysis in Tableau.
# Our team edited this so that correct dates were listed and the correct formatting was used.

# Give each week the starting day's date. 
two_weeks = datetime.timedelta(days=14)
week = datetime.timedelta(days=7)

# Get dates for 1st phase of surveys (every week starting 4/23)
week1 = datetime.datetime(2020, 4, 23)
phase1 = []
for i in range(0,13):
  phase1.append(str(week1.date()))
  week1 = week1 + week

# Get dates for 2nd phase of surveys (every two weeks starting 8/19)
week13 = datetime.datetime(2020, 8, 19)
phase2 = []
for i in range(0,16):
  phase2.append(str(week13.date()))
  week13 = week13 + two_weeks

weeks_date = phase1 + phase2
weeks_date.remove('2020-12-09')
weeks_date.remove('2020-04-30')
weeks_num = [i for i in range(1,28)]

# Save the 'DATE' variable back into the Dataframe with the changes that we have made.
data['DATE'] = data.WEEK

In [ ]:
# Next, our team focused in on the Vaccination variable in the dataset. As we were concerned with both vaccine sentiment and those who had already received a vaccine,
# we found a dynamic way to combine the variable that determined whether or not someone had received the vaccine with their vaccine sentiment.

# In the data, people were only asked about their sentiment on the vaccine (whether or not they would take the vaccine) if they had indicated that they had NOT already been vaccinated.
# Vaccinated indivduals were already vaccinated, and so their sentiments on the vaccine were already known.

# We combined these two variables into a single variable 'VACC', where the input was '9' if the individual was already vaccinated, and returned their sentiments on the vaccine otherwise.
data['VACC'] = np.where(data.RECVDVACC == 1, 9, data.GETVACC)

In [ ]:
# Next, certain survey responses were not ideal for EDA. For many variables, a number was listed that determined an individual's response, rather than their actual response. These 
# numbers were not always intuitive and often had multiple levels, making it necessary for our team to transform responses into text that we could interpret. This would allow our team
# to more easily discern relationships, and in performing EDA. 

variables = [{
  "col_name": "DATE",
  "answers": weeks_date,
  "values": weeks_num
}, {
  "col_name": "VACC",
  "answers": ["Definitely", "Probably", "Probably NOT", "Definitely NOT", "Vaccinated", np.nan, np.nan],
  "values": [ 1, 2, 3, 4, 9, -99, -88]
}, {
  "col_name": "HADCOVID",
  "answers": ['Yes', 'No', 'Not Sure', np.nan, np.nan],
  "values": [1,2,3,-99,-88]
}, {
  "col_name": "WRKLOSS",
  "answers":  ['Yes', 'No', np.nan, np.nan],
  "values": [1,2,-99,-88]
}, {
  "col_name": "KINDWORK",
  "answers": ["Government", "Private Company", "Non Profit Organization", 'Self-Employed', 'Working in a Family Business', np.nan, np.nan],
  "values": [1, 2, 3,4,5,-99,-88]
}, {
  "col_name": "TW_START",
  "answers": ["Yes", "No", "No Change", np.nan, np.nan],
  "values": [1, 2, 3,-99,-88]
}, {
  "col_name": "EXPNS_DIF",
  "answers": ["Not at all Difficult", "A little difficult", "Somewhat Difficult", 'Very Difficult', np.nan, np.nan],
  "values": [1, 2, 3,4,-99,-88]
}, {
  "col_name": "PLNDTRIPS",
  "answers": ["Yes", "No", np.nan, np.nan],
  "values": [1, 2, -99,-88]
}, {
  "col_name": "ANXIOUS",
  "answers": ["Not at all", "Several Days", "More than Half the Days", 'Nearly Every Day', np.nan, np.nan],
  "values": [1, 2, 3, 4, -99, -88]
}, {
  "col_name": "WORRY",
  "answers": ["Not at all", "Several Days", "More than Half the Days", 'Nearly Every Day', np.nan, np.nan],
  "values": [1, 2, 3, 4, -99, -88]
}, {
  "col_name": "INTEREST",
  "answers": ["Not at all", "Several Days", "More than Half the Days", 'Nearly Every Day', np.nan, np.nan],
  "values": [1, 2, 3, 4, -99, -88]
}, {
  "col_name": "DOWN",
  "answers": ["Not at all", "Several Days", "More than Half the Days", 'Nearly Every Day', np.nan, np.nan],
  "values": [1, 2, 3, 4, -99, -88]
}, {
  "col_name": "PRESCRIPT",
  "answers": ["Yes", "No", np.nan, np.nan],
  "values": [1, 2, -99,-88]
}, {
  "col_name": "MH_SVCS",
  "answers": ["Yes", "No", np.nan, np.nan],
  "values": [1, 2, -99,-88]
}, {
  "col_name": "MH_NOTGET",
  "answers": ["Yes", "No", np.nan, np.nan],
  "values": [1, 2, -99,-88]   
},{
  "col_name": "INCOME",
  "answers": ['Less than $25,000', '$25,000-$49,999', '$25,000-$49,999', '$50,000-74,999', '$75,000-$99,000','$100,000-$149,999',
            '$150,000 and above','$150,000 and above', np.nan, np.nan],
  "values": [1, 2, 3, 4, 5, 6, 7, 8, -99, -88]
}, {
  "col_name": "EST_ST",
  "answers":  [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
    'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
    'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
  "values": [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23,
            24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
            42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]
}, {
  "col_name": "PRIVHLTH",
  "answers": ["yes", "no", np.nan],
  "values": [1, 2, 3]
}, {
  "col_name": "PUBHLTH",
  "answers": ["yes", "no", np.nan],
  "values": [1, 2, 3]
}, {
  "col_name": "REGION",
  "answers": ["Northeast", "South", "Midwest", "West"],
  "values": [1, 2, 3, 4]
}]

# Next, our team defined a conditional response that ensured that the index and arrow choices were arrays of the same length. This would prevent 
# our transformations from changing rows in the Dataframe.
def categorize(col, indexes, choices):
  if len(indexes) != len(choices):
    return print('indexes and choices must be arrays of same length')
   
  conditions = [data[col] == index for index in indexes]
  return np.select(conditions, choices, default=np.nan)

In [ ]:
# For our listed variables, replace the values with the answers that we had indicated.
for i in variables:
  data[i['col_name']] = categorize(i['col_name'], i['values'], i['answers'])

In [ ]:
# Certain data responses were easier to change using the Pandas .loc function. This function finds all values within a specified column, and replaces them with
# values that our team has indicated.

# While birth year was listed in the dataset, our team found that analysis and relationships were easier to determine after breaking apart respondent age into age brackets.
# Using the respondent's birth year, we separated respondents into age brackets.
data['AGE_GROUP'] = ""
data.loc[(data.TBIRTH_YEAR >= 1996),'AGE_GROUP'] = "18 - 24"
data.loc[(data.TBIRTH_YEAR >= 1981) & (data.TBIRTH_YEAR <= 1995),'AGE_GROUP'] = "25 - 39"
data.loc[(data.TBIRTH_YEAR >= 1966) & (data.TBIRTH_YEAR <= 1980),'AGE_GROUP'] = "40 - 54"
data.loc[(data.TBIRTH_YEAR >= 1956) & (data.TBIRTH_YEAR <= 1965),'AGE_GROUP'] = "55 - 64"
data.loc[(data.TBIRTH_YEAR <= 1955),'AGE_GROUP'] = "65 and above"

# Our team transformed gender responses from 1 & 2 to Male and Female respectively.
data['GENDER'] = ""
data.loc[(data.EGENDER == 2),'GENDER'] = "Female"
data.loc[(data.EGENDER == 1),'GENDER'] = "Male"

# There were two variables that determined ethnicity in our dataset - one question prompted whether or not survey respondents were Hispanic, and another asked for their ethnicity.
# Our team combined these two variables into a single 'RACE_ETHNICITY' variable to analyze difference in vaccine sentiment between different indicated ethnicities.
data['RACE_ETHNICITY'] = ""
data.loc[data.RHISPANIC == 2,'RACE_ETHNICITY'] = "Hispanic"
data.loc[(data.RHISPANIC == 1) & (data.RRACE == 1),'RACE_ETHNICITY'] = "White alone"
data.loc[(data.RHISPANIC == 1) & (data.RRACE == 2),'RACE_ETHNICITY'] = "Black alone"
data.loc[(data.RHISPANIC == 1) & (data.RRACE == 3),'RACE_ETHNICITY'] = "Asian alone"
data.loc[(data.RHISPANIC == 1) & (data.RRACE == 4),'RACE_ETHNICITY'] = "Other races"

# Our team similarly transformed survey responses on Education level to text that we could use in analysis.
data['EDUCATION'] = ""
data.loc[(data.EEDUC >= 1) & (data.EEDUC <= 2),'EDUCATION'] = "Less than a high school diploma"
data.loc[(data.EEDUC == 3) ,'EDUCATION'] = "High school diploma or GED"
data.loc[(data.EEDUC >= 4) & (data.EEDUC <= 5),'EDUCATION'] = "Some college/associate's degree"
data.loc[(data.EEDUC >= 6) & (data.EEDUC <= 7),'EDUCATION'] = "Bachelor's degree or higher"

# Our team similarly transformed survey responses on Marital Status to divide survey respondents into 'Married' and 'Not Married' subgroups.
data['MARITAL_STATUS'] =""
data.loc[(data.MS == 1),'MARITAL_STATUS'] = "Married"
data.loc[(data.MS != 1),'MARITAL_STATUS'] = "Others"

# Our team extracted data on household size and added to our Pandas Dataframe.
data['HOUSEHOLD_SIZE'] = data['THHLD_NUMPER']

In [ ]:
# Next, the data provided 16 different reasons for which one was unlikely to take the vaccine. For each of these responses, our team transformed the responses to '1' indicating a 'Yes' 
# response to that question, and a '0' if the response was anything other than 1. This allowed our team to compare the reasons listed for not taking the vaccine.

data['YN1'] = ""
data.loc[(data.WHYNOT1 == 1), 'YN1'] = 1
data.loc[(data.WHYNOT1 != 1),'YN1'] = 0

data['YN2'] = ""
data.loc[(data.WHYNOT2 == 1), 'YN2'] = 1
data.loc[(data.WHYNOT2 != 1),'YN2'] = 0

data['YN3'] = ""
data.loc[(data.WHYNOT3 == 1), 'YN3'] = 1
data.loc[(data.WHYNOT3 != 1),'YN3'] = 0

data['YN4'] = ""
data.loc[(data.WHYNOT4 == 1), 'YN4'] = 1
data.loc[(data.WHYNOT4 != 1),'YN4'] = 0

data['YN5'] = ""
data.loc[(data.WHYNOT5 == 1), 'YN5'] = 1
data.loc[(data.WHYNOT5 != 1),'YN5'] = 0

data['YN6'] = ""
data.loc[(data.WHYNOT6 == 1), 'YN6'] = 1
data.loc[(data.WHYNOT6 != 1),'YN6'] = 0

data['YN7'] = ""
data.loc[(data.WHYNOT7 == 1), 'YN7'] = 1
data.loc[(data.WHYNOT7 != 1),'YN7'] = 0

data['YN8'] = ""
data.loc[(data.WHYNOT8 == 1), 'YN8'] = 1
data.loc[(data.WHYNOT8 != 1),'YN8'] = 0

data['YN9'] = ""
data.loc[(data.WHYNOT9 == 1), 'YN9'] = 1
data.loc[(data.WHYNOT9 != 1),'YN9'] = 0

data['YN10'] = ""
data.loc[(data.WHYNOT10 == 1), 'YN10'] = 1
data.loc[(data.WHYNOT10 != 1),'YN10'] = 0

data['YN11'] = ""
data.loc[(data.WHYNOT11 == 1), 'YN11'] = 1
data.loc[(data.WHYNOT11 != 1),'YN11'] = 0

data['YNB1'] = ""
data.loc[(data.WHYNOTB1 == 1), 'YNB1'] = 1
data.loc[(data.WHYNOTB1 != 1),'YNB1'] = 0

data['YNB2'] = ""
data.loc[(data.WHYNOTB2 == 1), 'YNB2'] = 1
data.loc[(data.WHYNOTB2 != 1),'YNB2'] = 0

data['YNB3'] = ""
data.loc[(data.WHYNOTB3 == 1), 'YNB3'] = 1
data.loc[(data.WHYNOTB3 != 1),'YNB3'] = 0

data['YNB4'] = ""
data.loc[(data.WHYNOTB4 == 1), 'YNB4'] = 1
data.loc[(data.WHYNOTB4 != 1),'YNB4'] = 0

data['YNB5'] = ""
data.loc[(data.WHYNOTB5 == 1), 'YNB5'] = 1
data.loc[(data.WHYNOTB5 != 1),'YNB5'] = 0

data['YNB6'] = ""
data.loc[(data.WHYNOTB6 == 1), 'YNB6'] = 1
data.loc[(data.WHYNOTB6 != 1),'YNB6'] = 0

# We next saved the responses for these columns into a column list that we would later add to our final dataframe.
why_cols = [ f'YN{num}' for num in range(1,12)] + [f'YNB{num}' for num in range(1,7)]

In [ ]:
# After extracting and transforming our variables of interest from our combined Dataframe, our team now combined the columns that we had transformed into 
# a single large dataframe for further analysis. This dataframe is 'fdata'. 

md_cols = [col['col_name'] for col in variables]
cols_keep = ['WEEK'] + subgroupList + why_cols+ md_cols 
fdata = data[cols_keep]

In [ ]:
# Show a preview of this dataframe to ensure that it looks correct. 
fdata

,WEEK,AGE_GROUP,GENDER,RACE_ETHNICITY,EDUCATION,MARITAL_STATUS,HOUSEHOLD_SIZE,RECVDVACC,GETVACC,YN1,YN2,YN3,YN4,YN5,YN6,YN7,YN8,YN9,YN10,YN11,YNB1,YNB2,YNB3,YNB4,YNB5,YNB6,DATE,VACC,HADCOVID,WRKLOSS,KINDWORK,TW_START,EXPNS_DIF,PLNDTRIPS,ANXIOUS,WORRY,INTEREST,DOWN,PRESCRIPT,MH_SVCS,MH_NOTGET,INCOME,EST_ST,PRIVHLTH,PUBHLTH,REGION
0,1,25 - 39,Female,White alone,Bachelor's degree or higher,Married,4,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-23,nan,nan,No,Private Company,nan,nan,nan,Nearly Every Day,More than Half the Days,Not at all,Not at all,nan,nan,nan,"$50,000-74,999",Tennessee,nan,nan,nan
1,1,65 and above,Male,White alone,Some college/associate's degree,Others,1,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-23,nan,nan,No,nan,nan,nan,nan,More than Half the Days,Nearly Every Day,Nearly Every Day,Nearly Every Day,nan,nan,nan,"$25,000-$49,999",Alabama,nan,nan,nan
2,1,40 - 54,Female,Other races,Bachelor's degree or higher,Married,2,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-23,nan,nan,No,Non Profit Organization,nan,nan,nan,Not at all,Not at all,Not at all,Not at all,nan,nan,nan,"$150,000 and above",Michigan,nan,nan,nan
3,1,55 - 64,Male,White alone,Some college/associate's degree,Others,2,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-23,nan,nan,Yes,nan,nan,nan,nan,Nearly Every Day,Nearly Every Day,Nearly Every Day,Nearly Every Day,nan,nan,nan,"Less than $25,000",Alabama,nan,nan,nan
4,1,55 - 64,Female,White alone,Bachelor's degree or higher,Others,1,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-04-23,nan,nan,No,Non Profit Organization,nan,nan,nan,Several Days,Not at all,Several Days,Several Days,nan,nan,nan,"$50,000-74,999",Alabama,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323332,27,40 - 54,Female,White alone,Bachelor's degree or higher,Married,2,2.0,4.0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,2021-03-17,Definitely NOT,No,Yes,Government,No Change,Very Difficult,Yes,Nearly Every Day,Nearly Every Day,Nearly Every Day,Nearly Every Day,No,No,Yes,"$75,000-$99,000",Wyoming,yes,no,West
2323333,27,40 - 54,Female,White alone,Some college/associate's degree,Married,4,1.0,-88.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-03-17,Vaccinated,No,No,Government,No,Somewhat Difficult,Yes,nan,nan,nan,nan,nan,nan,nan,nan,Wyoming,nan,nan,West
2323334,27,65 and above,Male,Hispanic,Bachelor's degree or higher,Married,2,1.0,-88.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-03-17,Vaccinated,No,No,Private Company,Yes,Not at all Difficult,Yes,Not at all,Not at all,Not at all,Not at all,No,No,No,"$100,000-$149,999",Wyoming,no,yes,West
2323335,27,55 - 64,Male,White alone,Some college/associate's degree,Married,2,1.0,-88.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-03-17,Vaccinated,No,No,nan,No,Not at all Difficult,Yes,Not at all,Not at all,Not at all,Not at all,No,No,No,"$50,000-74,999",Wyoming,yes,yes,West


In [ ]:
# Lastly, now that our final Dataframe is ready, our team saved this dataframe into a Parquet file that we could export to visualization software.

# File path that this parquet file was saved to was removed to maintain anonymity.
fdata.to_parquet('')

In [ ]:
# We also opted to save our final dataframe into a CSV file for use on external visualization software in our Shared Drive.

# File path that this CSV file was saved to was removed to maintain anonymity.
fdata.to_csv('')